In [1]:
import cv2

import os
import matplotlib.pyplot as plt
import pandas as pd

from efficientnet_pytorch import EfficientNet

In [2]:
efn = EfficientNet.from_name('efficientnet-b6', include_top = False)

In [3]:
efn

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 56, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(56, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        56, 56, kernel_size=(3, 3), stride=[1, 1], groups=56, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(56, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        56, 14, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        14, 56, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        56, 32, kernel_siz

# SENet

In [11]:
__all__ = ['SENet', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152',
           'se_resnext50_32x4d', 'se_resnext101_32x4d']

pretrained_settings = {
    'senet154': {
        'imagenet': {
            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/senet154-c7b49a05.pth',
            'input_space': 'RGB',
            'input_size': [3, 224, 224],
            'input_range': [0, 1],
            'mean': [0.485, 0.456, 0.406],
            'std': [0.229, 0.224, 0.225],
            'num_classes': 1000
        }
    },
    'se_resnet50': {
        'imagenet': {
            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/se_resnet50-ce0d4300.pth',
            'input_space': 'RGB',
            'input_size': [3, 224, 224],
            'input_range': [0, 1],
            'mean': [0.485, 0.456, 0.406],
            'std': [0.229, 0.224, 0.225],
            'num_classes': 1000
        }
    },
    'se_resnet101': {
        'imagenet': {
            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/se_resnet101-7e38fcc6.pth',
            'input_space': 'RGB',
            'input_size': [3, 224, 224],
            'input_range': [0, 1],
            'mean': [0.485, 0.456, 0.406],
            'std': [0.229, 0.224, 0.225],
            'num_classes': 1000
        }
    },
    'se_resnet152': {
        'imagenet': {
            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/se_resnet152-d17c99b7.pth',
            'input_space': 'RGB',
            'input_size': [3, 224, 224],
            'input_range': [0, 1],
            'mean': [0.485, 0.456, 0.406],
            'std': [0.229, 0.224, 0.225],
            'num_classes': 1000
        }
    },
    'se_resnext50_32x4d': {
        'imagenet': {
            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/se_resnext50_32x4d-a260b3a4.pth',
            'input_space': 'RGB',
            'input_size': [3, 224, 224],
            'input_range': [0, 1],
            'mean': [0.485, 0.456, 0.406],
            'std': [0.229, 0.224, 0.225],
            'num_classes': 1000
        }
    },
    'se_resnext101_32x4d': {
        'imagenet': {
            'url': 'http://data.lip6.fr/cadene/pretrainedmodels/se_resnext101_32x4d-3b2fe3d8.pth',
            'input_space': 'RGB',
            'input_size': [3, 224, 224],
            'input_range': [0, 1],
            'mean': [0.485, 0.456, 0.406],
            'std': [0.229, 0.224, 0.225],
            'num_classes': 1000
        }
    },
}

In [12]:
from __future__ import division, absolute_import
import math
from collections import OrderedDict
import torch.nn as nn
from torch.utils import model_zoo




class SEModule(nn.Module):

    def __init__(self, channels, reduction):
        super(SEModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(
            channels, channels // reduction, kernel_size=1, padding=0
        )
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Conv2d(
            channels // reduction, channels, kernel_size=1, padding=0
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        module_input = x
        x = self.avg_pool(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return module_input * x


class Bottleneck(nn.Module):
    """
    Base class for bottlenecks that implements `forward()` method.
    """

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out = self.se_module(out) + residual
        out = self.relu(out)

        return out


class SEBottleneck(Bottleneck):
    """
    Bottleneck for SENet154.
    """
    expansion = 4

    def __init__(
        self, inplanes, planes, groups, reduction, stride=1, downsample=None
    ):
        super(SEBottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes * 2, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes * 2)
        self.conv2 = nn.Conv2d(
            planes * 2,
            planes * 4,
            kernel_size=3,
            stride=stride,
            padding=1,
            groups=groups,
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(planes * 4)
        self.conv3 = nn.Conv2d(
            planes * 4, planes * 4, kernel_size=1, bias=False
        )
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.se_module = SEModule(planes * 4, reduction=reduction)
        self.downsample = downsample
        self.stride = stride
        

class SEResNetBottleneck(Bottleneck):
    """
    ResNet bottleneck with a Squeeze-and-Excitation module. It follows Caffe
    implementation and uses `stride=stride` in `conv1` and not in `conv2`
    (the latter is used in the torchvision implementation of ResNet).
    """
    expansion = 4

    def __init__(
        self, inplanes, planes, groups, reduction, stride=1, downsample=None
    ):
        super(SEResNetBottleneck, self).__init__()
        self.conv1 = nn.Conv2d(
            inplanes, planes, kernel_size=1, bias=False, stride=stride
        )
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(
            planes,
            planes,
            kernel_size=3,
            padding=1,
            groups=groups,
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.se_module = SEModule(planes * 4, reduction=reduction)
        self.downsample = downsample
        self.stride = stride


class SEResNeXtBottleneck(Bottleneck):
    """ResNeXt bottleneck type C with a Squeeze-and-Excitation module"""
    expansion = 4

    def __init__(
        self,
        inplanes,
        planes,
        groups,
        reduction,
        stride=1,
        downsample=None,
        base_width=4
    ):
        super(SEResNeXtBottleneck, self).__init__()
        width = int(math.floor(planes * (base_width/64.)) * groups)
        self.conv1 = nn.Conv2d(
            inplanes, width, kernel_size=1, bias=False, stride=1
        )
        self.bn1 = nn.BatchNorm2d(width)
        self.conv2 = nn.Conv2d(
            width,
            width,
            kernel_size=3,
            stride=stride,
            padding=1,
            groups=groups,
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(width)
        self.conv3 = nn.Conv2d(width, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.se_module = SEModule(planes * 4, reduction=reduction)
        self.downsample = downsample
        self.stride = stride

In [13]:
class SENet(nn.Module):
    """Squeeze-and-excitation network.
    
    Reference:
        Hu et al. Squeeze-and-Excitation Networks. CVPR 2018.
    Public keys:
        - ``senet154``: SENet154.
        - ``se_resnet50``: ResNet50 + SE.
        - ``se_resnet101``: ResNet101 + SE.
        - ``se_resnet152``: ResNet152 + SE.
        - ``se_resnext50_32x4d``: ResNeXt50 (groups=32, width=4) + SE.
        - ``se_resnext101_32x4d``: ResNeXt101 (groups=32, width=4) + SE.
        - ``se_resnet50_fc512``: (ResNet50 + SE) + FC.
    """

    def __init__(
        self,
        block,
        layers,
        groups,
        reduction,
        dropout_p=0.2,
        inplanes=128,
        input_3x3=True,
        downsample_kernel_size=3,
        downsample_padding=1,
        last_stride=2,
        **kwargs
    ):
        """
        Parameters
        ----------
        block (nn.Module): Bottleneck class.
            - For SENet154: SEBottleneck
            - For SE-ResNet models: SEResNetBottleneck
            - For SE-ResNeXt models:  SEResNeXtBottleneck
        layers (list of ints): Number of residual blocks for 4 layers of the
            network (layer1...layer4).
        groups (int): Number of groups for the 3x3 convolution in each
            bottleneck block.
            - For SENet154: 64
            - For SE-ResNet models: 1
            - For SE-ResNeXt models:  32
        reduction (int): Reduction ratio for Squeeze-and-Excitation modules.
            - For all models: 16
        dropout_p (float or None): Drop probability for the Dropout layer.
            If `None` the Dropout layer is not used.
            - For SENet154: 0.2
            - For SE-ResNet models: None
            - For SE-ResNeXt models: None
        inplanes (int):  Number of input channels for layer1.
            - For SENet154: 128
            - For SE-ResNet models: 64
            - For SE-ResNeXt models: 64
        input_3x3 (bool): If `True`, use three 3x3 convolutions instead of
            a single 7x7 convolution in layer0.
            - For SENet154: True
            - For SE-ResNet models: False
            - For SE-ResNeXt models: False
        downsample_kernel_size (int): Kernel size for downsampling convolutions
            in layer2, layer3 and layer4.
            - For SENet154: 3
            - For SE-ResNet models: 1
            - For SE-ResNeXt models: 1
        downsample_padding (int): Padding for downsampling convolutions in
            layer2, layer3 and layer4.
            - For SENet154: 1
            - For SE-ResNet models: 0
            - For SE-ResNeXt models: 0
        num_classes (int): Number of outputs in `classifier` layer.
        """
        super(SENet, self).__init__()
        self.inplanes = inplanes

        if input_3x3:
            layer0_modules = [
                (
                    'conv1',
                    nn.Conv2d(3, 64, 3, stride=2, padding=1, bias=False)
                ),
                ('bn1', nn.BatchNorm2d(64)),
                ('relu1', nn.ReLU(inplace=True)),
                (
                    'conv2',
                    nn.Conv2d(64, 64, 3, stride=1, padding=1, bias=False)
                ),
                ('bn2', nn.BatchNorm2d(64)),
                ('relu2', nn.ReLU(inplace=True)),
                (
                    'conv3',
                    nn.Conv2d(
                        64, inplanes, 3, stride=1, padding=1, bias=False
                    )
                ),
                ('bn3', nn.BatchNorm2d(inplanes)),
                ('relu3', nn.ReLU(inplace=True)),
            ]
        else:
            layer0_modules = [
                (
                    'conv1',
                    nn.Conv2d(
                        3,
                        inplanes,
                        kernel_size=7,
                        stride=2,
                        padding=3,
                        bias=False
                    )
                ),
                ('bn1', nn.BatchNorm2d(inplanes)),
                ('relu1', nn.ReLU(inplace=True)),
            ]
        # To preserve compatibility with Caffe weights `ceil_mode=True`
        # is used instead of `padding=1`.
        layer0_modules.append(
            ('pool', nn.MaxPool2d(3, stride=2, ceil_mode=True))
        )
        self.layer0 = nn.Sequential(OrderedDict(layer0_modules))
        self.layer1 = self._make_layer(
            block,
            planes=64,
            blocks=layers[0],
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=1,
            downsample_padding=0
        )
        self.layer2 = self._make_layer(
            block,
            planes=128,
            blocks=layers[1],
            stride=2,
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=downsample_kernel_size,
            downsample_padding=downsample_padding
        )
        self.layer3 = self._make_layer(
            block,
            planes=256,
            blocks=layers[2],
            stride=2,
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=downsample_kernel_size,
            downsample_padding=downsample_padding
        )
        self.layer4 = self._make_layer(
            block,
            planes=512,
            blocks=layers[3],
            stride=last_stride,
            groups=groups,
            reduction=reduction,
            downsample_kernel_size=downsample_kernel_size,
            downsample_padding=downsample_padding
        )

        self.global_avgpool = nn.AdaptiveAvgPool2d(1)


    def _make_layer(
        self,
        block,
        planes,
        blocks,
        groups,
        reduction,
        stride=1,
        downsample_kernel_size=1,
        downsample_padding=0
    ):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(
                    self.inplanes,
                    planes * block.expansion,
                    kernel_size=downsample_kernel_size,
                    stride=stride,
                    padding=downsample_padding,
                    bias=False
                ),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, groups, reduction, stride, downsample
            )
        )
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups, reduction))

        return nn.Sequential(*layers)


    def featuremaps(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

    def forward(self, x):
        f = self.featuremaps(x)
        v = self.global_avgpool(f)
        v = v.view(v.size(0), -1)
        
        return v



def init_pretrained_weights(model, model_url):
    """Initializes model with pretrained weights.
    
    Layers that don't match with pretrained layers in name or size are kept unchanged.
    """
    pretrain_dict = model_zoo.load_url(model_url)
    model_dict = model.state_dict()
    pretrain_dict = {
        k: v
        for k, v in pretrain_dict.items()
        if k in model_dict and model_dict[k].size() == v.size()
    }
    model_dict.update(pretrain_dict)
    model.load_state_dict(model_dict)




def se_resnext50_32x4d( pretrained=True, **kwargs):
    model = SENet(
        block=SEResNeXtBottleneck,
        layers=[3, 4, 6, 3],
        groups=32,
        reduction=16,
        dropout_p=None,
        inplanes=64,
        input_3x3=False,
        downsample_kernel_size=1,
        downsample_padding=0,
        last_stride=2,
        **kwargs
    )
    if pretrained:
        model_url = pretrained_settings['se_resnext50_32x4d']['imagenet']['url'
                                                                          ]
        init_pretrained_weights(model, model_url)
    return model


def se_resnext101_32x4d( pretrained=True, **kwargs):
    model = SENet(
        block=SEResNeXtBottleneck,
        layers=[3, 4, 23, 3],
        groups=32,
        reduction=16,
        dropout_p=None,
        inplanes=64,
        input_3x3=False,
        downsample_kernel_size=1,
        downsample_padding=0,
        last_stride=2,
        **kwargs
    )
    if pretrained:
        model_url = pretrained_settings['se_resnext101_32x4d']['imagenet'][
            'url']
        init_pretrained_weights(model, model_url)
    return model


    

In [14]:
print(se_resnext50_32x4d())

SENet(
  (layer0): Sequential(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  )
  (layer1): Sequential(
    (0): SEResNeXtBottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (se_module): SEModule(
     

In [18]:
model = se_resnext50_32x4d()

In [19]:
import torch

model(torch.zeros((8, 3, 512, 512))).shape

torch.Size([8, 2048])

In [20]:
class SEResNext50(nn.Module):
    def __init__(self, middle_layer = 1024, output_layer = 5):
        super(SEResNext50, self).__init__()
        self.senet = se_resnext50_32x4d()
        self.elu = nn.ELU()
        self.fc1 = nn.Linear(2048, middle_layer)
        self.fc2 = nn.Linear(middle_layer, output_layer)
        

    def forward(self, x):
        x = self.senet(x)
        x = self.fc1(x)
        x = self.elu(x)
        x = self.fc2(x)
        return x
    
    
model = SEResNext50(512, 5)

In [21]:
model(torch.zeros((8, 3, 512, 512)))

tensor([[-0.0213, -0.0576,  0.0745,  0.0843,  0.1075],
        [-0.0213, -0.0576,  0.0745,  0.0843,  0.1075],
        [-0.0213, -0.0576,  0.0745,  0.0843,  0.1075],
        [-0.0213, -0.0576,  0.0745,  0.0843,  0.1075],
        [-0.0213, -0.0576,  0.0745,  0.0843,  0.1075],
        [-0.0213, -0.0576,  0.0745,  0.0843,  0.1075],
        [-0.0213, -0.0576,  0.0745,  0.0843,  0.1075],
        [-0.0213, -0.0576,  0.0745,  0.0843,  0.1075]],
       grad_fn=<AddmmBackward>)

In [35]:
class EfnB6(nn.Module):
    
    def __init__(self, middle_layer = 1024, output_layer = 5):
        super(EfnB6, self).__init__()
        self.efn = EfficientNet.from_name('efficientnet-b6', include_top = False)
        self.elu = nn.ELU()
        self.fc1 = nn.Linear(2304, middle_layer)
        self.fc2 = nn.Linear(middle_layer, output_layer)
    
    def forward(self, x):
        x = self.efn(x)
        x = x.view(x.size(0), -1)
        
        x = self.fc1(x)
        x = self.elu(x)
        x = self.fc2(x)
        return x
    
# model = EfnB6()

In [22]:
import torch.nn.functional as F
from catalyst.contrib import registry
from torch import nn
import torch

def log_t(u, t):
    """Compute log_t for `u`."""

    if t == 1.0:
        return torch.log(u)
    else:
        return (u ** (1.0 - t) - 1.0) / (1.0 - t)


def exp_t(u, t):
    """Compute exp_t for `u`."""

    if t == 1.0:
        return torch.exp(u)
    else:
        return torch.relu(1.0 + (1.0 - t) * u) ** (1.0 / (1.0 - t))


def compute_normalization_fixed_point(activations, t, num_iters=5):
    """Returns the normalization value for each example (t > 1.0).
    Args:
    activations: A multi-dimensional tensor with last dimension `num_classes`.
    t: Temperature 2 (> 1.0 for tail heaviness).
    num_iters: Number of iterations to run the method.
    Return: A tensor of same rank as activation with the last dimension being 1.
    """

    mu = torch.max(activations, dim=-1).values.view(-1, 1)
    normalized_activations_step_0 = activations - mu

    normalized_activations = normalized_activations_step_0
    i = 0
    while i < num_iters:
        i += 1
        logt_partition = torch.sum(exp_t(normalized_activations, t), dim=-1).view(-1, 1)
        normalized_activations = normalized_activations_step_0 * (logt_partition ** (1.0 - t))

    logt_partition = torch.sum(exp_t(normalized_activations, t), dim=-1).view(-1, 1)

    return -log_t(1.0 / logt_partition, t) + mu


def compute_normalization(activations, t, num_iters=5):
    """Returns the normalization value for each example.
    Args:
    activations: A multi-dimensional tensor with last dimension `num_classes`.
    t: Temperature 2 (< 1.0 for finite support, > 1.0 for tail heaviness).
    num_iters: Number of iterations to run the method.
    Return: A tensor of same rank as activation with the last dimension being 1.
    """

    if t < 1.0:
        return None # not implemented as these values do not occur in the authors experiments...
    else:
        return compute_normalization_fixed_point(activations, t, num_iters)


def tempered_softmax(activations, t, num_iters=5):
    """Tempered softmax function.
    Args:
    activations: A multi-dimensional tensor with last dimension `num_classes`.
    t: Temperature tensor > 0.0.
    num_iters: Number of iterations to run the method.
    Returns:
    A probabilities tensor.
    """

    if t == 1.0:
        normalization_constants = torch.log(torch.sum(torch.exp(activations), dim=-1))
    else:
        normalization_constants = compute_normalization(activations, t, num_iters)

    return exp_t(activations - normalization_constants, t)


def bi_tempered_logistic_loss(activations, labels, t1, t2, label_smoothing=0.0, num_iters=5, num_classes = 5):

    """Bi-Tempered Logistic Loss with custom gradient.
    Args:
    activations: A multi-dimensional tensor with last dimension `num_classes`.
    labels: A tensor with shape and dtype as activations.
    t1: Temperature 1 (< 1.0 for boundedness).
    t2: Temperature 2 (> 1.0 for tail heaviness, < 1.0 for finite support).
    label_smoothing: Label smoothing parameter between [0, 1).
    num_iters: Number of iterations to run the method.
    Returns:
    A loss tensor.
    """
    target = F.one_hot(labels, num_classes).float()

    if label_smoothing > 0.0:
        target = (1 - num_classes / (num_classes - 1) * label_smoothing) * target + label_smoothing / (num_classes - 1)

    probabilities = tempered_softmax(activations, t2, num_iters)

    temp1 = (log_t(target + 1e-10, t1) - log_t(probabilities, t1)) * target
    temp2 = (1 / (2 - t1)) * (torch.pow(target, 2 - t1) - torch.pow(probabilities, 2 - t1))
    loss_values = temp1 - temp2

    return torch.sum(torch.sum(loss_values, dim=-1))
   
    
@registry.Criterion
class LabelSmoothingLoss(nn.Module):
    def __init__(self, smooth_factor=0.05):
        super().__init__()
        self.smooth_factor = smooth_factor

    def _smooth_labels(self, num_classes, target):
        # When label smoothing is turned on,
        # KL-divergence between q_{smoothed ground truth prob.}(w)
        # and p_{prob. computed by model}(w) is minimized.
        # If label smoothing value is set to zero, the loss
        # is equivalent to NLLLoss or CrossEntropyLoss.
        # All non-true labels are uniformly set to low-confidence.

        target_one_hot = F.one_hot(target, num_classes).float()
        target_one_hot[target_one_hot == 1] = 1 - self.smooth_factor
        target_one_hot[target_one_hot == 0] = self.smooth_factor
        return target_one_hot

    def forward(self, input, target):
        logp = F.log_softmax(input, dim=1)
        target_one_hot = self._smooth_labels(input.size(1), target)
        return F.kl_div(logp, target_one_hot, reduction='sum')

@registry.Criterion
class TemperedLogLoss(nn.Module):
    def __init__(self, label_smoothing=0.05, t1 = 0.7, t2 = 2, num_iters = 5):
        super().__init__()
        self.label_smoothing = label_smoothing
        self.t1 = t1
        self.t2 = t2
        self.num_iters = num_iters

    def forward(self, input, target):
        return bi_tempered_logistic_loss(input, target, self.t1, self.t2, self.label_smoothing, self.num_iters)

In [23]:
from albumentations.pytorch import ToTensor

def get_super_light_augmentations(image_size):
    return A.Compose([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
    ])

def get_light_augmentations(image_size):
    min_size = min(image_size[0], image_size[1])
    return A.Compose([
        A.RandomSizedCrop(min_max_height=(int(min_size * 0.85), min_size),
                          height=image_size[0],
                          width=image_size[1], p=1.0),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
    ])

def get_medium_augmentations(image_size):
    min_size = min(image_size[0], image_size[1])

    return A.Compose([
        A.OneOf([A.RandomSizedCrop(min_max_height=(int(min_size* 0.85), min_size),
                          height=image_size[0],
                          width=image_size[1]),
                A.Resize(image_size[0], image_size[1]),
                A.CenterCrop(image_size[0], image_size[1])
                ], p = 1.0),

        A.OneOf([
            A.RandomBrightnessContrast(brightness_limit=0.15,
                                       contrast_limit=0.5),
            A.RandomGamma(gamma_limit=(50, 150)),
            A.NoOp()
        ], p = 1.0),
        
        A.OneOf([A.CLAHE(p=0.5, clip_limit=(10, 10), tile_grid_size=(3, 3)),
                A.FancyPCA(alpha=0.4),
                A.NoOp(),
                ], p = 1.0),
        
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Flip(p = 0.5)
    ])

def get_hard_augmentations(image_size):
    return None

# def get_medium_augmentations(image_size):
#     return A.Compose([
#         A.OneOf([
#             A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1,
#                                rotate_limit=15,
#                                border_mode=cv2.BORDER_CONSTANT, value=0),
#             A.OpticalDistortion(distort_limit=0.11, shift_limit=0.15,
#                                 border_mode=cv2.BORDER_CONSTANT,
#                                 value=0),
#             A.NoOp()
#         ]),
#         A.RandomSizedCrop(min_max_height=(int(image_size[0] * 0.75), image_size[0]),
#                           height=image_size[0],
#                           width=image_size[1], p=0.3),
#         A.OneOf([
#             A.RandomBrightnessContrast(brightness_limit=0.5,
#                                        contrast_limit=0.4),
#             A.RandomGamma(gamma_limit=(50, 150)),
#             A.NoOp()
#         ]),
#         A.OneOf([
#             A.RGBShift(r_shift_limit=20, b_shift_limit=15, g_shift_limit=15),
#             A.HueSaturationValue(hue_shift_limit=5,
#                                  sat_shift_limit=5),
#             A.NoOp()
#         ]),
#         A.HorizontalFlip(p=0.5),
#         A.VerticalFlip(p=0.5)
#     ])

def post_transforms():
    # we use ImageNet image normalization
    # and convert it to torch.Tensor
#     return [ToTensor()]
    return A.Compose([A.Normalize(mean=(0.485, 0.456, 0.406),
                       std=(0.229, 0.224, 0.225)), ToTensor()])

# def compose(transforms_to_compose):
#     # combine all augmentations into one single pipeline
#     result = A.Compose([
#       item for sublist in transforms_to_compose for item in sublist
#     ])
#     return result


def get_test_transform(image_size):
    return A.Compose([A.Resize(image_size[0], image_size[1], p = 1.0), A.Normalize(mean=(0.485, 0.456, 0.406),
                       std=(0.229, 0.224, 0.225), p = 1.0), ToTensor()])

def get_train_transform(augmentation, image_size):
    LEVELS = {
        'super_light': get_super_light_augmentations,
        'light': get_light_augmentations,
        'medium': get_medium_augmentations,
        'hard': get_hard_augmentations,
#         'hard2': get_hard_augmentations_v2
    }

    aug = LEVELS[augmentation](image_size)

    return A.Compose([aug, post_transforms()])

In [24]:
import math
import os
from typing import Tuple, List

import albumentations as A
import cv2
import numpy as np
import pandas as pd
# from pytorch_toolbelt.utils import fs
# from pytorch_toolbelt.utils.fs import id_from_fname
# from pytorch_toolbelt.utils.torch_utils import tensor_from_rgb_image
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.utils import compute_sample_weight
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler





UNLABELED_CLASS = -100



class TaskDataset(Dataset):
    def __init__(self, images, targets, img_size, 
                 transform: A.Compose,
                 target_as_array=False,
                 dtype=int):
        if targets is not None:
            targets = np.array(targets)
            unique_targets = set(targets)
            if len(unique_targets.difference({0, 1, 2, 3, 4, UNLABELED_CLASS})):
                raise ValueError('Unexpected targets in Y ' + str(unique_targets))

        self.images = np.array(images)
        self.targets = targets
        self.transform = transform
        self.target_as_array = target_as_array
        self.dtype = dtype
        self.img_size = img_size

    def __len__(self):
        return len(self.images)

    def __getitem__(self, item):
        image = cv2.imread(self.images[item])  # Read with OpenCV instead PIL. It's faster
        if image is None:
            raise FileNotFoundError(self.images[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        height, width = image.shape[:2]
        label = UNLABELED_CLASS

        if self.targets is not None:
            label = self.targets[item]

        #Плохо тут это делать но пока так
        data = self.transform(image=image, label=label)
        label = data['label']

        data = {'image': data['image']}

        label = self.dtype(label)
        if self.target_as_array:
            data['targets'] = np.array([label])
        else:
            data['targets'] = label

        return data





def split_train_valid(x, y, fold=None, folds=4, random_state=42):
    """
    Common train/test split function
    :param x:
    :param y:
    :param fold:
    :param folds:
    :param random_state:
    :return:
    """
    train_x, train_y = [], []
    valid_x, valid_y = [], []

    if fold is not None:
        assert 0 <= fold < folds
        skf = StratifiedKFold(n_splits=folds, random_state=random_state, shuffle=True)

        for fold_index, (train_index, test_index) in enumerate(skf.split(x, y)):
            if fold_index == fold:
                train_x = x[train_index]
                train_y = y[train_index]
                valid_x = x[test_index]
                valid_y = y[test_index]
                break
    else:
        train_x, valid_x, train_y, valid_y = train_test_split(x, y,
                                                              random_state=random_state,
                                                              test_size=1.0 / folds,
                                                              shuffle=True,
                                                              stratify=y)

    assert len(train_x) and len(train_y) and len(valid_x) and len(valid_y)
    assert len(train_x) == len(train_y)
    assert len(valid_x) == len(valid_y)
    return train_x, valid_x, train_y, valid_y


def get_current_train(data_dir):
    df = pd.read_csv(os.path.join(data_dir, 'train_images.csv'))
    x = np.array(df['image_id'].apply(lambda x: os.path.join(data_dir, 'train_images', f'{x}')))
    y = np.array(df['label'], dtype=int)
    return x, y


def get_extra_train(data_dir):
    df = pd.read_csv(os.path.join(data_dir, 'extra_images.csv'))
    x = np.array(df['image_id'].apply(lambda x: os.path.join(data_dir, 'extra_data', f'{x}')))
    y = np.array(df['label'], dtype=int)
    return x, y


def get_unlabeled(dataset_dir, healthy_eye_fraction=1):
    df= pd.read_csv(os.path.join(dataset_dir, 'unlabeled.csv'))
    x = np.array(df['image_id'].apply(lambda x: os.path.join(data_dir, 'unlabeled', f'{x}')))
    y = np.array([UNLABELED_CLASS] * len(x), dtype=int)
    return x, y






def append_train_test(existing, to_add):
    train_x, train_y, valid_x, valid_y = existing
    tx, vx, ty, vy = to_add
    train_x.extend(tx)
    train_y.extend(ty)
    valid_x.extend(vx)
    valid_y.extend(vy)
    return train_x, train_y, valid_x, valid_y


def get_dataset(datasets: List[str], data_dir='data', random_state=42):
    """
    :param datasets: List "aptos2015-train/fold0", "messidor",
    :param fold:
    :param folds:
    :return:
    """

    all_x = []
    all_y = []
    sizes = []

    for ds in datasets:

        dataset_name = ds

        if dataset_name == 'current_train':
            x, y = get_current_train(data_dir)
        elif dataset_name == 'extra_train':
            x, y = get_extra_train(data_dir)
        elif dataset_name == 'unlabeled':
            x, y = get_aptos2015_test(data_dir)
        else:
            raise ValueError(dataset_name)

        all_x.extend(x)
        all_y.extend(y)
        sizes.append(len(x))

    return all_x, all_y, sizes


def get_datasets_universal(
        train_on: List[str],
        valid_on: List[str],
        data_dir='data',
        image_size=(512, 512),
        augmentation='medium',
        target_dtype=int,
        random_state=42,
        coarse_grading=False,
        folds=4) -> Tuple[TaskDataset, TaskDataset, List]:
    train_x, train_y, sizes = get_dataset(train_on, data_dir=data_dir, random_state=random_state)
    valid_x, valid_y, _ = get_dataset(valid_on, data_dir=data_dir, random_state=random_state)

    train_transform = get_train_transform(augmentation = augmentation, image_size = image_size)
    valid_transform = get_test_transform(image_size = image_size)

    train_ds = TaskDataset(train_x, train_y, image_size,
                                  transform=train_transform,
                                  dtype=target_dtype)

    valid_ds = TaskDataset(valid_x, valid_y, image_size,
                                  transform=valid_transform,
                                  dtype=target_dtype)

    return train_ds, valid_ds, sizes


def get_datasets(
        data_dir='data',
        image_size=(512, 512),
        augmentation='medium',
        use_current=True,
        use_extra=False,
        use_unlabeled=False,
        target_dtype=int,
        random_state=42,
        fold=None,
        folds=4) -> Tuple[TaskDataset, TaskDataset, List]:
    assert use_current or use_extra or use_unlabeled

    trainset_sizes = []
    data_split = [], [], [], []

  

    if use_current:
        x, y = get_current_train(data_dir)
        split = split_train_valid(x, y, fold=fold, folds=folds, random_state=random_state)
        data_split = append_train_test(data_split, split)
        trainset_sizes.append(len(split[0]))

    if use_extra:
        x, y = get_extra_train(data_dir)
        split = split_train_valid(x, y, fold=fold, folds=folds, random_state=random_state)
        data_split = append_train_test(data_split, split)
        trainset_sizes.append(len(split[0]))

    if use_unlabeled:
        x, y = get_unlabeled(data_dir)
        split = split_train_valid(x, y, fold=fold, folds=folds, random_state=random_state)
        data_split = append_train_test(data_split, split)
        trainset_sizes.append(len(split[0]))

    train_x, train_y, valid_x, valid_y = data_split



    train_transform = get_train_transform(augmentation = augmentation, image_size = image_size)
    valid_transform = get_test_transform(image_size)

    train_ds = TaskDataset(train_x, train_y, image_size, 
                                      transform=train_transform,
                                      dtype=target_dtype)

    valid_ds = TaskDataset(valid_x, valid_y, image_size, 
                                  transform=valid_transform,
                                  dtype=target_dtype)

    return train_ds, valid_ds, trainset_sizes


def get_dataloaders(train_ds, valid_ds,
                    batch_size,
                    num_workers = 1,
                    fast=False,
                    train_sizes=None,
                    balance=False,
                    balance_datasets=False,
                    balance_unlabeled=False,
                    ):
    sampler = None
    weights = None
    num_samples = 0

    if balance_unlabeled:
        labeled_mask = (train_ds.targets != UNLABELED_CLASS).astype(np.uint8)
        weights = compute_sample_weight('balanced', labeled_mask)
        num_samples = int(np.mean(train_sizes))

    if balance:
        weights = compute_sample_weight('balanced', train_ds.targets)
        hist = np.bincount(train_ds.targets)
        min_class_counts = int(min(hist))
        num_classes = len(np.unique(train_ds.targets))
        num_samples = min_class_counts * num_classes

    if balance_datasets:
        assert train_sizes is not None
        dataset_balancing_term = []

        for subset_size in train_sizes:
            full_dataset_size = float(sum(train_sizes))
            dataset_balancing_term.extend([full_dataset_size / subset_size] * subset_size)

        dataset_balancing_term = np.array(dataset_balancing_term)
        if weights is None:
            weights = np.ones(len(train_ds.targets))

        weights = weights * dataset_balancing_term
        num_samples = int(np.mean(train_sizes))

    # If we do balancing, let's go for fixed number of batches (half of dataset)
    if weights is not None:
        sampler = WeightedRandomSampler(weights, num_samples)

    if fast:
        weights = np.ones(len(train_ds))
        sampler = WeightedRandomSampler(weights, 16)

    train_dl = DataLoader(train_ds, batch_size=batch_size,
                          shuffle=sampler is None, sampler=sampler,
                          pin_memory=True, drop_last=True)
    valid_dl = DataLoader(valid_ds, batch_size=batch_size, shuffle=False,
                          pin_memory=True, drop_last=False)

    return train_dl, valid_dl

In [25]:
train_ds, valid_ds, train_sizes = get_datasets(data_dir='../data',
                                                    use_current=True,
                                                    use_extra=False,
                                                    image_size=(28, 28),  
                                                    augmentation='medium',
                                                    target_dtype=int,
                                                    fold=1,
                                                    folds=4)

In [26]:
valid_ds[0]['image'].shape

torch.Size([3, 28, 28])

In [27]:
len(train_ds)

16048

In [28]:
train_loader, valid_loader = get_dataloaders(train_ds, valid_ds,
                                                batch_size=128,
                                                train_sizes=train_sizes,
                                                num_workers = 1,
                                                balance=True,
                                                balance_datasets=True,
                                                balance_unlabeled=False)

In [29]:
import collections
from catalyst.dl import SupervisedRunner, EarlyStoppingCallback
from catalyst.utils import load_checkpoint, unpack_checkpoint

loaders = collections.OrderedDict()
loaders["train"] = train_loader
loaders["valid"] = valid_loader

runner = SupervisedRunner(input_key='image')

In [30]:
import catalyst 
from catalyst.dl import AccuracyCallback

# criterions = LabelSmoothingLoss()
criterions = TemperedLogLoss()
# optimizer = catalyst.contrib.nn.optimizers.radam.RAdam(model.parameters(), lr = learning_rate)
optimizer = catalyst.contrib.nn.optimizers.Adam(model.parameters(), lr = 1e-3)
# criterions = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# cappa = CappaScoreCallback()
callbacks = [AccuracyCallback(num_classes=5)]

main_metric = 'accuracy01'

runner.train(
    fp16=True,
    model=model,
    criterion=criterions,
    optimizer=optimizer,
    callbacks=callbacks,
    loaders=loaders,
    num_epochs=10,
    verbose=1,
    main_metric=main_metric,
    minimize_metric=False,
)

1/10 * Epoch (train): 100% 125/125 [02:17<00:00,  1.10s/it, accuracy01=0.289, accuracy03=0.703, loss=105.665]
Early exiting                                                                                             
1/10 * Epoch (valid):  50% 21/42 [00:18<00:18,  1.15it/s, accuracy01=0.586, accuracy03=0.852, loss=75.257]

In [ ]:
next(iter(valid_loader))['image']

In [ ]:
a = model(next(iter(valid_loader))['image'])

# Количество параметров

SEResNext50

In [32]:

seresnext50_model = SEResNext50(512, 5)
sum(p.numel() for p in seresnext50_model.parameters())

26562549

EfficientNetB6

In [36]:
efnB6_model = EfnB6(512, 5)
sum(p.numel() for p in efnB6_model.parameters())

44223429